In [21]:
from PIL import Image
import os
import math
import matplotlib.pyplot as plt

def pad_image(image, patch_size=256, overlap=64):
    """将图像填充到基于步长的尺寸"""
    width, height = image.size
    step = patch_size - overlap  # 192
    # 计算需要的列数和行数
    cols = math.ceil((width - patch_size) / step) + 1  # ceil((1001-256)/192)+1=5
    rows = math.ceil((height - patch_size) / step) + 1  # ceil((565-256)/192)+1=3
    # 填充尺寸
    target_width = (cols - 1) * step + patch_size  # 4*192 + 256 = 1024
    target_height = (rows - 1) * step + patch_size  # 2*192 + 256 = 640
    
    padded = Image.new('RGB', (target_width, target_height), (255, 255, 255))
    x_offset = (target_width - width) // 2  # (1024-1001)//2=11
    y_offset = (target_height - height) // 2  # (640-565)//2=37
    padded.paste(image, (x_offset, y_offset))
    
    return padded, (x_offset, y_offset, x_offset + width, y_offset + height), (target_width, target_height)

def split_image(image, target_size, original_coords, patch_size=256, overlap=64):
    """分割图像，考虑重叠，取消 overlap_area"""
    target_width, target_height = target_size
    patches = []
    coords = []
    step = patch_size - overlap  # 192
    
    grid_cols = math.ceil((target_width - patch_size) / step) + 1  # 5
    grid_rows = math.ceil((target_height - patch_size) / step) + 1  # 3
    
    for i in range(grid_rows):
        for j in range(grid_cols):
            x_start = j * step
            y_start = i * step
            x_end = min(x_start + patch_size, target_width)
            y_end = min(y_start + patch_size, target_height)
            
            patch = image.crop((x_start, y_start, x_end, y_end))
            if patch.size != (patch_size, patch_size):
                patch = patch.resize((patch_size, patch_size), Image.Resampling.BILINEAR)
            
            patches.append(patch)
            coords.append((x_start, y_start, x_end, y_end))
    
    return patches, coords, (grid_rows, grid_cols)

def split_and_save_image(image_path, output_dir, patch_size=256, overlap=64):
    """主函数：填充并分割图像"""
    try:
        image = Image.open(image_path).convert('RGB')
        padded_image, original_coords, target_size = pad_image(image, patch_size, overlap)
        print(f"原始尺寸: {image.size}, 填充后尺寸: {target_size[0]}x{target_size[1]}")
        print(f"原始图像坐标: {original_coords}")
        
        patches, coords, (grid_rows, grid_cols) = split_image(padded_image, target_size, original_coords, patch_size, overlap)
        
        os.makedirs(output_dir, exist_ok=True)
        fig, axes = plt.subplots(grid_rows, grid_cols, figsize=(grid_cols*4, grid_rows*4))
        axes = axes.flatten() if grid_rows * grid_cols > 1 else [axes]
        
        for ax in axes:
            ax.axis('off')
        
        for i, (patch, (x_start, y_start, x_end, y_end)) in enumerate(zip(patches, coords)):
            patch_path = os.path.join(output_dir, f"img5patch_{i+1}.png")
            patch.save(patch_path)
            print(f"子图 {i+1} 已保存到 {patch_path}")
            
            ax = axes[i]
            ax.imshow(patch)
            ax.set_title(f"子图 {i+1}\n({x_start},{y_start})-({x_end},{y_end})", fontsize=8)
            ax.axis('off')
        
        vis_path = os.path.join(output_dir, "patches_vis.png")
        plt.tight_layout()
        plt.savefig(vis_path, dpi=300, bbox_inches='tight')
        plt.close(fig)
        print(f"子图可视化已保存到 {vis_path}")
        
        return patches, coords, original_coords, target_size
    
    except Exception as e:
        print(f"运行时发生错误: {e}")
        return None, None, None, None

if __name__ == "__main__":
    input_image = r"C:\Users\27805\Desktop\bin\test\paperimage\红11\img5\origimage.jpg"
    output_dir = r"C:\Users\27805\Desktop\bin\test\origpatchs"
    patches, coords, original_coords, target_size = split_and_save_image(input_image, output_dir)
    if patches:
        print(f"分割坐标: {coords}")
        print(f"原始图像坐标: {original_coords}")
        print(f"目标尺寸: {target_size[0]}x{target_size[1]}")
        print(f"分割成 {len(patches)} 份")

原始尺寸: (580, 579), 填充后尺寸: 640x640
原始图像坐标: (30, 30, 610, 609)
子图 1 已保存到 C:\Users\27805\Desktop\bin\test\origpatchs\img5patch_1.png
子图 2 已保存到 C:\Users\27805\Desktop\bin\test\origpatchs\img5patch_2.png
子图 3 已保存到 C:\Users\27805\Desktop\bin\test\origpatchs\img5patch_3.png
子图 4 已保存到 C:\Users\27805\Desktop\bin\test\origpatchs\img5patch_4.png
子图 5 已保存到 C:\Users\27805\Desktop\bin\test\origpatchs\img5patch_5.png
子图 6 已保存到 C:\Users\27805\Desktop\bin\test\origpatchs\img5patch_6.png
子图 7 已保存到 C:\Users\27805\Desktop\bin\test\origpatchs\img5patch_7.png
子图 8 已保存到 C:\Users\27805\Desktop\bin\test\origpatchs\img5patch_8.png
子图 9 已保存到 C:\Users\27805\Desktop\bin\test\origpatchs\img5patch_9.png


C:\Users\27805\AppData\Local\Temp\ipykernel_60384\3766053128.py:78: UserWarning: Glyph 23376 (\N{CJK UNIFIED IDEOGRAPH-5B50}) missing from current font.
  plt.tight_layout()
C:\Users\27805\AppData\Local\Temp\ipykernel_60384\3766053128.py:78: UserWarning: Glyph 22270 (\N{CJK UNIFIED IDEOGRAPH-56FE}) missing from current font.
  plt.tight_layout()
C:\Users\27805\AppData\Local\Temp\ipykernel_60384\3766053128.py:79: UserWarning: Glyph 23376 (\N{CJK UNIFIED IDEOGRAPH-5B50}) missing from current font.
  plt.savefig(vis_path, dpi=300, bbox_inches='tight')
C:\Users\27805\AppData\Local\Temp\ipykernel_60384\3766053128.py:79: UserWarning: Glyph 22270 (\N{CJK UNIFIED IDEOGRAPH-56FE}) missing from current font.
  plt.savefig(vis_path, dpi=300, bbox_inches='tight')


子图可视化已保存到 C:\Users\27805\Desktop\bin\test\origpatchs\patches_vis.png
分割坐标: [(0, 0, 256, 256), (192, 0, 448, 256), (384, 0, 640, 256), (0, 192, 256, 448), (192, 192, 448, 448), (384, 192, 640, 448), (0, 384, 256, 640), (192, 384, 448, 640), (384, 384, 640, 640)]
原始图像坐标: (30, 30, 610, 609)
目标尺寸: 640x640
分割成 9 份


In [16]:
from PIL import Image
import os
import numpy as np
import math

def infer_grid_size(num_patches):
    """
    根据子图片数量推断网格的行数和列数，优先选择宽布局（列数 >= 行数）。
    """
    sqrt_num = math.ceil(math.sqrt(num_patches))
    for cols in range(sqrt_num, num_patches + 1):
        if num_patches % cols == 0:
            rows = num_patches // cols
            # 优先选择列数 >= 行数的布局（例如 3x4 而不是 4x3）
            if cols >= rows:
                return rows, cols
    cols = sqrt_num
    rows = math.ceil(num_patches / cols)
    return rows, cols

def merge_images(input_folder, output_path, original_size=None, patch_size=256, overlap=64):
    """
    将子图片合并成一张大图。
    
    参数：
    input_folder: 子图片所在的文件夹
    output_path: 合并后图片的保存路径
    original_size: 原始图片的尺寸 (width, height)，如果提供则裁剪到此尺寸
    patch_size: 子图片的尺寸（默认 256）
    overlap: 分割时的重叠像素（默认 64）
    """
    # 获取子图片列表，适配 img2patch_ 前缀
    patch_files = sorted([f for f in os.listdir(input_folder) if f.startswith('img6patch_') and f.endswith('.png')],
                         key=lambda x: int(x.split('_')[1].split('.')[0]))
    num_patches = len(patch_files)
    if num_patches == 0:
        print("错误：文件夹中没有找到子图片！")
        return
    
    # 推断网格大小
    grid_rows, grid_cols = infer_grid_size(num_patches)
    print(f"推断网格大小：{grid_rows} 行 x {grid_cols} 列")
    
    # 计算填充后图片的尺寸
    step = patch_size - overlap  # 步长考虑重叠
    target_width = grid_cols * step + overlap
    target_height = grid_rows * step + overlap
    print(f"推断填充后尺寸：{target_width}x{target_height}")
    
    # 创建空白画布和累加数组（使用 float32）
    merged_image_array = np.zeros((target_height, target_width, 3), dtype=np.float32)
    count_map = np.zeros((target_height, target_width, 3), dtype=np.float32)
    
    # 遍历子图片
    for i, patch_file in enumerate(patch_files):
        patch_path = os.path.join(input_folder, patch_file)
        patch = Image.open(patch_path).convert('RGB')
        
        # 确保子图片尺寸正确
        if patch.size != (patch_size, patch_size):
            print(f"警告：子图片 {patch_file} 尺寸 {patch.size} 与预期 {patch_size}x{patch_size} 不符")
            patch = patch.resize((patch_size, patch_size), Image.Resampling.BILINEAR)
        
        # 计算子图片在网格中的位置（按行优先）
        row = i // grid_cols
        col = i % grid_cols
        x_start = col * step
        y_start = row * step
        x_end = x_start + patch_size
        y_end = y_start + patch_size
        
        # 转换为 numpy 数组
        patch_array = np.array(patch, dtype=np.float32)
        
        # 粘贴到合并图像
        for y in range(y_start, min(y_end, target_height)):
            for x in range(x_start, min(x_end, target_width)):
                patch_y = y - y_start
                patch_x = x - x_start
                if patch_y < patch_array.shape[0] and patch_x < patch_array.shape[1]:
                    merged_image_array[y, x] += patch_array[patch_y, patch_x]
                    count_map[y, x] += 1
    
    # 处理重叠区域（取平均值）
    count_map[count_map == 0] = 1  # 避免除以零
    merged_image_array = merged_image_array / count_map
    merged_image_array = np.clip(merged_image_array, 0, 255).astype(np.uint8)
    merged_image = Image.fromarray(merged_image_array)
    
    # 如果提供了原始尺寸，裁剪掉填充区域
    if original_size:
        original_width, original_height = original_size
        x1 = (target_width - original_width) // 2
        y1 = (target_height - original_height) // 2
        x2 = x1 + original_width
        y2 = y1 + original_height
        merged_image = merged_image.crop((x1, y1, x2, y2))
    
    # 保存合并后的图片
    merged_image.save(output_path)
    print(f"合并完成，保存至 {output_path}")

# 示例用法
if __name__ == "__main__":
    # 配置参数
    input_folder = r"C:\Users\27805\Desktop\bin\test\patches"  # 子图片所在文件夹
    output_path = r"C:\Users\27805\Desktop\bin\test\paperimage\红6\img6\ourimage.png"  # 合并后图片保存路径
    original_size = None  # 替换为原始图片的尺寸（例如 (768, 768)），如果不知道设为 None
    
    merge_images(input_folder, output_path, original_size)

推断网格大小：3 行 x 5 列
推断填充后尺寸：1024x640
合并完成，保存至 C:\Users\27805\Desktop\bin\test\paperimage\红6\img6\ourimage.png


In [ ]:
from PIL import Image

def crop_image_region(image_path, output_path, top_left, bottom_right):
    """
    从图片中裁剪指定区域。
    
    参数：
    image_path: 输入图片路径
    output_path: 输出裁剪后图片路径
    top_left: 左上角坐标 (x1, y1)
    bottom_right: 右下角坐标 (x2, y2)
    """
    # 打开图片
    img = Image.open(image_path)
    
    # 提取坐标
    x1, y1 = top_left
    x2, y2 = bottom_right
    
    # 裁剪区域 (left, upper, right, lower)
    crop_area = (x1, y1, x2, y2)
    
    # 裁剪图片
    cropped_img = img.crop(crop_area)
    
    # 保存裁剪后的图片
    cropped_img.save(output_path)
    print(f"裁剪后的图片已保存到 {output_path}")

# 示例使用
image_path = r"C:\Users\27805\Desktop\bin\test\paperimage\img6\ourimagesim.png"  # 替换为你的图片路径
output_path = r"C:\Users\27805\Desktop\bin\test\paperimage\img6\ourimagedetail.png"  # 替换为输出图片路径
w=500#左右
h=180#上下
top_left = (w, h)  # 替换为左上角坐标 (x1, y1)
bottom_right = (w+170*892/586, h+170)  # 替换为右下角坐标 (x2, y2)
#bottom_right = (w+303, h+170)  # 替换为右下角坐标 (x2, y2)
crop_image_region(image_path, output_path, top_left, bottom_right)

In [17]:
from PIL import Image

def crop_white_borders(original_size, enhanced_image_path, output_path):
    # 原始图片尺寸
    original_width, original_height = original_size
    
    # 打开增强图片
    enhanced_img = Image.open(enhanced_image_path)
    
    # 获取增强图片尺寸
    enhanced_width, enhanced_height = enhanced_img.size
    
    # 计算白边宽度和高度（假设白边对称）
    border_width = (enhanced_width - original_width) // 2
    border_height = (enhanced_height - original_height) // 2
    
    # 计算裁剪区域 (left, upper, right, lower)
    left = border_width
    upper = border_height
    right = enhanced_width - border_width
    lower = enhanced_height - border_height
    
    # 裁剪图片
    cropped_img = enhanced_img.crop((left, upper, right, lower))
    
    # 保存裁剪后的图片
    cropped_img.save(output_path)
    print(f"裁剪后的图片已保存到 {output_path}")

# 示例使用
original_size = (891, 586)  # 替换为原始图片的宽度和高度，例如 (1920, 1080)
enhanced_image_path = r"C:\Users\27805\Desktop\bin\test\paperimage\红6\img6\ourimage.png"  # 替换为增强图片的路径
output_path = r"C:\Users\27805\Desktop\bin\test\paperimage\红6\img6\ourimage.png"  # 替换为输出图片的路径

crop_white_borders(original_size, enhanced_image_path, output_path)

裁剪后的图片已保存到 C:\Users\27805\Desktop\bin\test\paperimage\红6\img6\ourimage.png
